In [3]:
from tqdm import tqdm
import numpy as np
from scipy.linalg import fractional_matrix_power
from scipy.linalg import expm

/home/cesc/Desktop/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Definition of the NKTgp

In [4]:
sigma_w = np.sqrt(1.5)
sigma_b = 1.
Mu = 0.
STD = 1.
input_size = 1
eta = 0.1
n0 = 1
depth = 3

time = 1000

For the activation function $\phi = {\rm erf}$, we have:

In [66]:
def mathT(a, b, c, d):
    return 2./np.pi * np.arcsin( 2.* b / np.sqrt((1+2*a)*(1+2*d)))

def mathTp(a, b, c, d):
    #det = a*c - b*d
    auxMat = np.empty((2,2))
    auxMat[0,0] = a
    auxMat[0,1] = b
    auxMat[1,0] = c
    auxMat[1,1] = d
    return 4./ np.pi * np.linalg.det(fractional_matrix_power(np.eye(2) + 2*auxMat, -1./2.))
    #return 4./ np.pi / (np.linalg.det(np.eye(2) + 2*auxMat))**2.

For the activation function ReLU $\phi(x) = \max(x, 0)$, we have:

In [6]:
def mathT(a, b, c, d):
    a = np.linalg.norm(a)
    d = np.linalg.norm(d)
    fact_or = b/np.sqrt(a*d)
    output = np.sin(np.arccos(fact_or)) + (np.pi - np.arccos(fact_or))*fact_or
    return 1./2./np.pi * np.sqrt(a*d) * output


def mathTp(a, b, c, d):
    a = np.linalg.norm(a)
    d = np.linalg.norm(d)
    return 1./2./np.pi * (np.pi - np.arccos(b/np.sqrt(a*d)))

In [7]:
def Kappa(arr1, arr2, l_index, sigma_w, sigma_b, n0):
    if l_index == 1:
        return np.dot(arr1,arr2)/n0*sigma_w**2. + sigma_b**2.
    else:
        T11 = Kappa(arr1, arr1, l_index-1, sigma_w, sigma_b, n0)
        T12 = Kappa(arr1, arr2, l_index-1, sigma_w, sigma_b, n0)
        T21 = T12
        T22 = Kappa(arr2, arr2, l_index-1, sigma_w, sigma_b, n0)
        return sigma_w**2.*mathT(T11, T12, T21, T22) + sigma_b**2.

    
    
def Theta(arr1, arr2, l_index, sigma_w, sigma_b, n0):
    if l_index == 1:
        return np.dot(arr1,arr2)/n0*sigma_w**2. + sigma_b**2.
    else:
        T11 = Kappa(arr1, arr1, l_index-1, sigma_w, sigma_b, n0)
        T12 = Kappa(arr1, arr2, l_index-1, sigma_w, sigma_b, n0)
        T21 = T12
        T22 = Kappa(arr2, arr2, l_index-1, sigma_w, sigma_b, n0)
        return sigma_w**2.*mathTp(T11, T12, T21, T22)*Theta(arr1, arr2, l_index-1, sigma_w, sigma_b, n0)

In [68]:
def distribNKTgp(time, x, input_train, output_train, eta, n_layer, sigma_w, sigma_b, n0):
    tdim = len(input_train)
    Kxx = Kappa(x, x, n_layer+1, sigma_w, sigma_b, n0)
    KxCalX = np.empty((tdim))
    TxCalX = np.empty((tdim))
    KCalXCalX = np.empty((tdim, tdim))
    TCalXCalX = np.empty((tdim, tdim))
    
    for index, elem in enumerate(input_train):
        KxCalX[index] = Kappa(x, elem, n_layer+1, sigma_w, sigma_b, n0)
        TxCalX[index] = Theta(x, elem, n_layer+1, sigma_w, sigma_b, n0)
        for jndex in range(index,tdim):
            KCalXCalX[index, jndex] = Kappa(elem, input_train[jndex], n_layer+1, sigma_w, sigma_b, n0)
            KCalXCalX[jndex, index] = KCalXCalX[index, jndex]
            TCalXCalX[index, jndex] = Theta(elem, input_train[jndex], n_layer+1, sigma_w, sigma_b, n0)
            TCalXCalX[jndex, index] = TCalXCalX[index, jndex]

    T_inverse = np.linalg.inv(TCalXCalX)
    Aux = np.eye(tdim) - expm(-eta*TCalXCalX*time)
    
    mean = np.matmul(TxCalX, np.matmul(T_inverse, np.matmul(Aux, output_train)))
    variance = Kxx + np.matmul(TxCalX, np.matmul(T_inverse, np.matmul(Aux, np.matmul(KCalXCalX, np.matmul(Aux, np.matmul(T_inverse, TxCalX))))))
    vaux = np.matmul(TxCalX, np.matmul(T_inverse, np.matmul(Aux, KxCalX)))
    variance -= vaux + np.conj(vaux)
    return mean, np.sqrt(abs(variance))

# Initial steps for the NKT Ising

### Data Preparation

+ N = # of spins
+ g = transverse field
+ h = longitudinal field
+ Mx = longitudinal Magnetization
+ energy 

In [69]:
N, g, h, Mx, energy = np.loadtxt("../data/data.dat", unpack=True)

In [70]:
num_train = len(N) - 1
num_test = len(N) - num_train

input = np.ndarray(shape = (len(N), 3))
for ind, elem in enumerate(input):
    input[ind] = np.array([N[ind], g[ind], h[ind]])

input_train = input[0:num_train,:]
input_test = input[len(N)-1,:]
for ind, elem in enumerate(input):
    if ind == num_train:
        print(elem)

Mxout_train = Mx[0:num_train]
Mxout_test = Mx[num_train:len(N)]

print(input_test, Mxout_test)

[10.   1.8  0.5]
[10.   1.8  0.5] [0.66771903]


### Application of the NKTgp algorithm
the right way is to use the activation function $\phi = {\rm erf}\,\,$:

In [73]:
sigma_b = 3
sigma_w = 2

n0 = 3
time = 10000
eta = 0.1
n_layer = 3

print(distribNKTgp(time, input_test, input_train, Mxout_train, eta, n_layer, sigma_w, sigma_b, n0))

(0.676300985115283, 1.0100117012698748)
